# ML

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

drivers = [
    "/opt/spark/external-jars/hadoop-aws-3.3.4.jar",             # S3
    "/opt/spark/external-jars/aws-java-sdk-bundle-1.12.262.jar", # S3
    "/opt/spark/external-jars/wildfly-openssl-1.0.7.Final.jar",  # S3
    "/opt/spark/external-jars/postgresql-42.6.0.jar",            # PostgreSQL
]

spark = (SparkSession.builder
         .appName("mustdayker-Spark")
         .master("spark://spark-master:7077") 
         .config("spark.jars", ",".join(drivers))
         .getOrCreate()
        )

25/11/19 20:50:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [ ]:
# Блок 1: Импорт библиотек и инициализация Spark
from pyspark.sql.types import *

In [2]:
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder

from pyspark.ml.classification import GBTClassifier
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import BinaryClassificationEvaluator, RegressionEvaluator

from pyspark.ml import Pipeline
import time

In [3]:
df = spark.read.parquet("s3a://bronze/nyc-taxi-data/yellow_tripdata_2025-07.parquet") #yellow_tripdata_2025-09/")

25/11/19 21:04:39 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
                                                                                

In [ ]:
df = spark.read.parquet("s3a://silver/nyc-taxi-data-norm/*") #yellow_tripdata_2025-09/")

In [57]:
df = spark.read.parquet("s3a://silver/nyc-taxi-data-eda/yellow_tripdata_2025-09") #yellow_tripdata_2025-09/")

In [ ]:
spark.stop()

План выполнения кода:
- Блок 1: Импорт библиотек и инициализация Spark
- Блок 2: Загрузка и первичный осмотр данных
- Блок 3: Подготовка целевых переменных и создание признаков (Feature Engineering)
- Блок 4: Подготовка фичей для ML (векторизация, индексация)
- Блок 5: Разделение на тренировочную и тестовую выборки (70/30)
- Блок 6: Модель классификации - Gradient Boosting (GBTClassifier)
- Блок 7: Модель регрессии - Gradient Boosting (GBTRegressor)
- Блок 8: Оценка моделей и создание результатов для дашборда
- Блок 9: Интерпретация метрик (как читать результаты)

# Блок 2: Загрузка и первичный осмотр данных

In [4]:
# Блок 2: Загрузка и первичный осмотр данных
# Предполагаем, что данные уже заджойнены с информацией о районах
df = spark.read.parquet("s3a://silver/nyc-taxi-data-eda/yellow_tripdata_2025-09")

# Посмотрим на структуру данных
print("=== Схема данных ===")
df.printSchema()

print("=== Количество строк ===")
print(f"Всего записей: {df.count():,}")

print("=== Первые 5 строк ===")
df.show(1)

# Проверим основные статистики по числовым колонкам
print("=== Основные статистики ===")
df.describe("trip_distance", "fare_amount", "tip_amount", "total_amount", "passenger_count").show()

=== Схема данных ===
root
 |-- vendorid: integer (nullable = true)
 |-- vendor_name: string (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- ratecodeid: integer (nullable = true)
 |-- ratecode_name: string (nullable = true)
 |-- pulocationid: integer (nullable = true)
 |-- pickup_borough: string (nullable = true)
 |-- pickup_zone: string (nullable = true)
 |-- dolocationid: integer (nullable = true)
 |-- dropoff_borough: string (nullable = true)
 |-- dropoff_zone: string (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- payment_name: string (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge

25/11/19 21:04:55 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Всего записей: 3,784,669
=== Первые 5 строк ===
+--------+--------------------+--------------------+---------------------+---------------+-------------+----------+-------------+------------+--------------+------------+------------+---------------+-------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+---------------------+-------------------+----+-----+---+-----------+----+----------+-----------+------------+-----------------+-------------------+-------+------------------+
|vendorid|         vendor_name|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|ratecodeid|ratecode_name|pulocationid|pickup_borough| pickup_zone|dolocationid|dropoff_borough| dropoff_zone|payment_type|payment_name|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|trip_duration_minutes|         date_month|year|month|day|day_o

## Блок 3: Подготовка целевых переменных и создание признаков (Feature Engineering)

In [5]:
# Блок 3: Подготовка целевых переменных и создание признаков (Feature Engineering)

# Создаем целевую переменную для классификации - наличие чаевых
df = df.withColumn("has_tip", F.when(F.col("tip_amount") > 0, 1).otherwise(0))

# Создаем временные признаки из даты пикапа
df = df.withColumn("hour", F.hour("tpep_pickup_datetime")) \
       .withColumn("day_of_week", F.dayofweek("tpep_pickup_datetime")) \
       .withColumn("is_weekend", 
                  F.when((F.col("day_of_week") == 1) | (F.col("day_of_week") == 7), 1).otherwise(0)) \
       .withColumn("month", F.month("tpep_pickup_datetime"))

# Создаем признак длительности поездки в минутах
df = df.withColumn("trip_duration_minutes", 
                  (F.unix_timestamp("tpep_dropoff_datetime") - F.unix_timestamp("tpep_pickup_datetime")) / 60)

# Создаем признак скорости поездки (миль в час)
df = df.withColumn("speed_mph", 
                  F.when(F.col("trip_duration_minutes") > 0, 
                       F.col("trip_distance") / (F.col("trip_duration_minutes") / 60))
                  .otherwise(0))

# Создаем признак поездки в аэропорт
# JFK = 132, LGA = 138, EWR = 1 (нужно уточнить по вашим данным)
df = (
    df.withColumn("is_airport_pickup",  F.when(F.col("PULocationID").isin([132, 138, 1]), 1).otherwise(0))
      .withColumn("is_airport_dropoff", F.when(F.col("DOLocationID").isin([132, 138, 1]), 1).otherwise(0))
      .withColumn("is_airport_trip",    F.when((F.col("is_airport_pickup") == 1) | (F.col("is_airport_dropoff") == 1), 1).otherwise(0))
     )

# Проверим созданные признаки
print("=== Проверка созданных признаков ===")
(df.sample(fraction=0.1)
 .select(["has_tip", "hour", "day_of_week", "is_weekend", "trip_duration_minutes", "speed_mph", "is_airport_trip"])
 .show(10))

print("=== Распределение целевых переменных ===")
print("Классификация (чаевые):")
df.groupBy("has_tip").count().show()

print("Регрессия (стоимость):")
df.describe("total_amount",).show()

=== Проверка созданных признаков ===
+-------+----+-----------+----------+---------------------+------------------+---------------+
|has_tip|hour|day_of_week|is_weekend|trip_duration_minutes|         speed_mph|is_airport_trip|
+-------+----+-----------+----------+---------------------+------------------+---------------+
|      1|   0|          2|         0|   10.466666666666667|12.038216560509554|              0|
|      1|   0|          2|         0|   25.966666666666665| 36.50834403080873|              1|
|      1|   0|          2|         0|                 13.1| 12.82442748091603|              0|
|      1|   0|          2|         0|   24.983333333333334|33.382254836557706|              1|
|      1|   0|          2|         0|                 9.75|11.076923076923077|              0|
|      1|   0|          2|         0|   15.183333333333334| 18.17782656421515|              0|
|      1|   0|          2|         0|   16.083333333333332| 9.326424870466322|              0|
|      0|   0

# Блок 4: Подготовка фичей для ML (векторизация, индексация)

In [6]:
# Блок 4: Подготовка фичей для ML (векторизация, индексация)

# Определяем категориальные и числовые признаки
categorical_cols = ["hour", "day_of_week", "pickup_borough", "dropoff_borough", "is_weekend", "is_airport_trip"]
numeric_cols = ["trip_distance", "passenger_count", "trip_duration_minutes", "speed_mph"]

# Создаем stages для пайплайна
stages = []

In [7]:
# Обрабатываем категориальные переменные через StringIndexer + OneHotEncoder
for categorical_col in categorical_cols:
    # Создаем StringIndexer для каждой категориальной колонки
    string_indexer = StringIndexer(inputCol=categorical_col, outputCol=categorical_col + "_index")
    # Создаем OneHotEncoder для преобразования в числовой формат
    encoder = OneHotEncoder(inputCol=categorical_col + "_index", outputCol=categorical_col + "_encoded")
    stages += [string_indexer, encoder]

# Собираем все обработанные фичи в один вектор
assembler_inputs = [c + "_encoded" for c in categorical_cols] + numeric_cols

# VectorAssembler объединяет все признаки в один векторный столбец
feature_assembler = VectorAssembler(inputCols=assembler_inputs, outputCol="features")
stages += [feature_assembler]

# Создаем пайплайн для обработки данных
preprocessing_pipeline = Pipeline(stages=stages)

# Применяем пайплайн к данным
print("=== Запуск пайплайна предобработки ===")
start_time = time.time()
preprocessing_model = preprocessing_pipeline.fit(df)
df_processed = preprocessing_model.transform(df)
processing_time = time.time() - start_time

print(f"Пайплайн выполнен за {processing_time:.2f} секунд")
print("=== Данные после обработки ===")
df_processed.select("features", "has_tip", "total_amount").show(5, truncate=False)

=== Запуск пайплайна предобработки ===


Пайплайн выполнен за 5.38 секунд
=== Данные после обработки ===
+------------------------------------------------------------------------------------------------------------+-------+------------+
|features                                                                                                    |has_tip|total_amount|
+------------------------------------------------------------------------------------------------------------+-------+------------+
|(49,[16,27,29,36,43,44,45,46,47,48],[1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,12.716666666666667,9.43643512450852])  |1      |23.95       |
|(49,[16,27,29,36,43,44,45,46,47,48],[1.0,1.0,1.0,1.0,1.0,1.0,3.1,2.0,10.2,18.23529411764706])               |1      |24.75       |
|(49,[16,27,29,36,43,44,45,46,47,48],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,11.966666666666667,5.013927576601671]) |1      |21.4        |
|(49,[16,27,29,36,43,44,45,46,47,48],[1.0,1.0,1.0,1.0,1.0,1.0,5.6,1.0,13.933333333333334,24.114832535885167])|0      |29.75       |
|(49,[16,27,

# Блок 5: Разделение на тренировочную и тестовую выборки (70/30)

In [8]:
# Блок 5: Разделение на тренировочную и тестовую выборки (70/30)

# Для классификации используем стратифицированную выборку для сохранения баланса классов
print("=== Разделение данных для классификации (чаевые) ===")
classifier_df = df_processed.select("features", "has_tip")

# Стратифицированное разделение для классификации
classifier_train, classifier_test = classifier_df.randomSplit([0.7, 0.3], seed=42)

print(f"Тренировочные данные: {classifier_train.count():,} записей")
print(f"Тестовые данные: {classifier_test.count():,} записей")

# Проверяем распределение классов в обеих выборках
print("Распределение классов в тренировочной выборке:")
classifier_train.groupBy("has_tip").count().withColumn("percentage", 
                                                      F.col("count") / classifier_train.count() * 100).show()

print("Распределение классов в тестовой выборке:")
classifier_test.groupBy("has_tip").count().withColumn("percentage", 
                                                     F.col("count") / classifier_test.count() * 100).show()

# Для регрессии простое разделение
print("\n=== Разделение данных для регрессии (стоимость) ===")
regression_df = df_processed.select("features", "total_amount")
regression_train, regression_test = regression_df.randomSplit([0.7, 0.3], seed=42)

print(f"Тренировочные данные: {regression_train.count():,} записей")
print(f"Тестовые данные: {regression_test.count():,} записей")

=== Разделение данных для классификации (чаевые) ===


Тренировочные данные: 2,648,670 записей


Тестовые данные: 1,135,999 записей
Распределение классов в тренировочной выборке:


+-------+-------+------------------+
|has_tip|  count|        percentage|
+-------+-------+------------------+
|      1|1695469| 64.01208908622064|
|      0| 953201|35.987910913779366|
+-------+-------+------------------+

Распределение классов в тестовой выборке:


+-------+------+-----------------+
|has_tip| count|       percentage|
+-------+------+-----------------+
|      1|726590|63.96044362715108|
|      0|409409|36.03955637284892|
+-------+------+-----------------+


=== Разделение данных для регрессии (стоимость) ===


Тренировочные данные: 2,648,670 записей


[Stage 59:===================================================>     (9 + 1) / 10]

Тестовые данные: 1,135,999 записей


# Блок 6: Модель классификации - Gradient Boosting (GBTClassifier)

In [9]:
# Блок 6: Модель классификации - Gradient Boosting (GBTClassifier)

print("=== ЗАПУСК МОДЕЛИ КЛАССИФИКАЦИИ ===")
print("Алгоритм: Gradient Boosted Trees Classifier")
print("Задача: Предсказание наличия чаевых (has_tip)")

start_time = time.time()

# Создаем модель Gradient Boosting для классификации
gbt_classifier = GBTClassifier(
    featuresCol="features",
    labelCol="has_tip",
    maxIter=100,           # Количество деревьев (увеличиваем для лучшей точности)
    maxDepth=6,            # Глубина деревьев
    stepSize=0.1,          # Скорость обучения
    subsamplingRate=0.8,   # Доля данных для каждого дерева
    seed=42
)

print("Обучение модели классификации...")
gbt_model = gbt_classifier.fit(classifier_train)
training_time_classifier = time.time() - start_time

print(f"Обучение завершено за {training_time_classifier:.2f} секунд")

# Делаем предсказания на тестовых данных
print("Прогнозирование на тестовых данных...")
predictions_classifier = gbt_model.transform(classifier_test)

print("=== Пример предсказаний ===")
predictions_classifier.select("has_tip", "prediction", "probability").show(10)

# Оцениваем модель
evaluator_binary = BinaryClassificationEvaluator(
    labelCol="has_tip",
    rawPredictionCol="rawPrediction"
)

evaluator_auc = BinaryClassificationEvaluator(
    labelCol="has_tip",
    rawPredictionCol="rawPrediction",
    metricName="areaUnderROC"
)

# Вычисляем различные метрики
auc = evaluator_auc.evaluate(predictions_classifier)
areaUnderPR = evaluator_binary.setMetricName("areaUnderPR").evaluate(predictions_classifier)

print("\n=== МЕТРИКИ КЛАССИФИКАЦИИ ===")
print(f"Area Under ROC (AUC): {auc:.4f}")
print(f"Area Under PR: {areaUnderPR:.4f}")

# Дополнительные метрики через MulticlassClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator_multi = MulticlassClassificationEvaluator(
    labelCol="has_tip",
    predictionCol="prediction"
)

accuracy = evaluator_multi.setMetricName("accuracy").evaluate(predictions_classifier)
f1 = evaluator_multi.setMetricName("f1").evaluate(predictions_classifier)
weightedPrecision = evaluator_multi.setMetricName("weightedPrecision").evaluate(predictions_classifier)
weightedRecall = evaluator_multi.setMetricName("weightedRecall").evaluate(predictions_classifier)

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Precision: {weightedPrecision:.4f}")
print(f"Recall: {weightedRecall:.4f}")

=== ЗАПУСК МОДЕЛИ КЛАССИФИКАЦИИ ===
Алгоритм: Gradient Boosted Trees Classifier
Задача: Предсказание наличия чаевых (has_tip)
Обучение модели классификации...


25/11/19 21:11:05 WARN DAGScheduler: Broadcasting large task binary with size 1003.4 KiB
25/11/19 21:11:05 WARN DAGScheduler: Broadcasting large task binary with size 1007.3 KiB
25/11/19 21:11:06 WARN DAGScheduler: Broadcasting large task binary with size 1007.9 KiB
25/11/19 21:11:07 WARN DAGScheduler: Broadcasting large task binary with size 1008.6 KiB
25/11/19 21:11:07 WARN DAGScheduler: Broadcasting large task binary with size 1009.7 KiB
25/11/19 21:11:07 WARN DAGScheduler: Broadcasting large task binary with size 1012.1 KiB
25/11/19 21:11:08 WARN DAGScheduler: Broadcasting large task binary with size 1016.9 KiB
25/11/19 21:11:08 WARN DAGScheduler: Broadcasting large task binary with size 1020.5 KiB
25/11/19 21:11:09 WARN DAGScheduler: Broadcasting large task binary with size 1021.0 KiB
25/11/19 21:11:10 WARN DAGScheduler: Broadcasting large task binary with size 1021.7 KiB
25/11/19 21:11:10 WARN DAGScheduler: Broadcasting large task binary with size 1022.8 KiB
25/11/19 21:11:10 WAR

Обучение завершено за 325.72 секунд
Прогнозирование на тестовых данных...
=== Пример предсказаний ===


25/11/19 21:13:05 WARN DAGScheduler: Broadcasting large task binary with size 1507.5 KiB
25/11/19 21:13:05 WARN DAGScheduler: Broadcasting large task binary with size 1507.5 KiB
25/11/19 21:13:14 WARN DAGScheduler: Broadcasting large task binary with size 1506.3 KiB


+-------+----------+--------------------+
|has_tip|prediction|         probability|
+-------+----------+--------------------+
|      0|       1.0|[0.40243590998732...|
|      0|       1.0|[0.26651848380179...|
|      0|       0.0|[0.63413704028473...|
|      0|       0.0|[0.63413704028473...|
|      0|       0.0|[0.63413704028473...|
|      1|       1.0|[0.33744573306960...|
|      1|       1.0|[0.33744573306960...|
|      0|       1.0|[0.41356870559301...|
|      1|       1.0|[0.41356870559301...|
|      1|       0.0|[0.63413704028473...|
+-------+----------+--------------------+
only showing top 10 rows



25/11/19 21:13:28 WARN DAGScheduler: Broadcasting large task binary with size 1506.3 KiB
                                                                                


=== МЕТРИКИ КЛАССИФИКАЦИИ ===
Area Under ROC (AUC): 0.8423
Area Under PR: 0.8650


25/11/19 21:13:45 WARN DAGScheduler: Broadcasting large task binary with size 1518.1 KiB
25/11/19 21:14:00 WARN DAGScheduler: Broadcasting large task binary with size 1518.1 KiB
25/11/19 21:14:15 WARN DAGScheduler: Broadcasting large task binary with size 1518.1 KiB
25/11/19 21:14:30 WARN DAGScheduler: Broadcasting large task binary with size 1518.1 KiB
[Stage 1300:=================================================>     (9 + 1) / 10]

Accuracy: 0.8439
F1 Score: 0.8363
Precision: 0.8525
Recall: 0.8439


# Блок 7: Модель регрессии - Gradient Boosting (GBTRegressor)

In [81]:
# Блок 7: Модель регрессии - Gradient Boosting (GBTRegressor)

print("=== ЗАПУСК МОДЕЛИ РЕГРЕССИИ ===")
print("Алгоритм: Gradient Boosted Trees Regressor")
print("Задача: Предсказание общей стоимости поездки (total_amount)")

start_time = time.time()

# Создаем модель Gradient Boosting для регрессии
gbt_regressor = GBTRegressor(
    featuresCol="features",
    labelCol="total_amount",
    maxIter=100,           # Количество деревьев
    maxDepth=6,            # Глубина деревьев
    stepSize=0.1,          # Скорость обучения
    subsamplingRate=0.8,   # Доля данных для каждого дерева
    seed=42
)

print("Обучение модели регрессии...")
gbt_regressor_model = gbt_regressor.fit(regression_train)
training_time_regression = time.time() - start_time

print(f"Обучение завершено за {training_time_regression:.2f} секунд")

# Делаем предсказания на тестовых данных
print("Прогнозирование на тестовых данных...")
predictions_regression = gbt_regressor_model.transform(regression_test)

print("=== Пример предсказаний ===")
predictions_regression.select("total_amount", "prediction").show(10)

# Оцениваем модель регрессии
evaluator_regression = RegressionEvaluator(
    labelCol="total_amount",
    predictionCol="prediction"
)

# Вычисляем различные метрики регрессии
rmse = evaluator_regression.setMetricName("rmse").evaluate(predictions_regression)
mse = evaluator_regression.setMetricName("mse").evaluate(predictions_regression)
mae = evaluator_regression.setMetricName("mae").evaluate(predictions_regression)
r2 = evaluator_regression.setMetricName("r2").evaluate(predictions_regression)

print("\n=== МЕТРИКИ РЕГРЕССИИ ===")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"R² Score: {r2:.4f}")

=== ЗАПУСК МОДЕЛИ РЕГРЕССИИ ===
Алгоритм: Gradient Boosted Trees Regressor
Задача: Предсказание общей стоимости поездки (total_amount)
Обучение модели регрессии...


25/11/17 21:25:51 WARN DAGScheduler: Broadcasting large task binary with size 1002.2 KiB
25/11/17 21:25:51 WARN DAGScheduler: Broadcasting large task binary with size 1006.9 KiB
25/11/17 21:25:52 WARN DAGScheduler: Broadcasting large task binary with size 1010.6 KiB
25/11/17 21:25:53 WARN DAGScheduler: Broadcasting large task binary with size 1011.1 KiB
25/11/17 21:25:53 WARN DAGScheduler: Broadcasting large task binary with size 1011.8 KiB
25/11/17 21:25:54 WARN DAGScheduler: Broadcasting large task binary with size 1012.8 KiB
25/11/17 21:25:54 WARN DAGScheduler: Broadcasting large task binary with size 1015.2 KiB
25/11/17 21:25:55 WARN DAGScheduler: Broadcasting large task binary with size 1020.0 KiB
25/11/17 21:25:55 WARN DAGScheduler: Broadcasting large task binary with size 1023.5 KiB
25/11/17 21:25:56 WARN DAGScheduler: Broadcasting large task binary with size 1024.0 KiB
25/11/17 21:25:56 WARN DAGScheduler: Broadcasting large task binary with size 1024.8 KiB
25/11/17 21:25:57 WAR

Обучение завершено за 342.78 секунд
Прогнозирование на тестовых данных...
=== Пример предсказаний ===


+------------+------------------+
|total_amount|        prediction|
+------------+------------------+
|       11.65| 14.33978122240012|
|       14.45|16.913286316765497|
|       11.65| 13.98311839539094|
|       11.65| 13.98311839539094|
|       11.65| 13.98311839539094|
|       18.85| 20.63465443848769|
|       21.54| 20.63465443848769|
|       27.71|13.782609791915558|
|       13.95|13.782609791915558|
|        13.1| 13.98311839539094|
+------------+------------------+
only showing top 10 rows



[Stage 2784:=================================================>     (9 + 1) / 10]


=== МЕТРИКИ РЕГРЕССИИ ===
Root Mean Squared Error (RMSE): 4.8500
Mean Squared Error (MSE): 23.5228
Mean Absolute Error (MAE): 2.7828
R² Score: 0.9331


In [83]:
# Вычисляем Mean Absolute Percentage Error (MAPE) вручную

predictions_with_mape = predictions_regression.withColumn(
    "ape", 
    F.when(F.col("total_amount") != 0, 
         F.abs(F.col("prediction") - F.col("total_amount")) / F.col("total_amount") * 100)
    .otherwise(0)
)

mape = predictions_with_mape.select(F.avg("ape")).collect()[0][0]
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")

[Stage 2786:=================================================>     (9 + 1) / 10]

Mean Absolute Percentage Error (MAPE): 9.98%


# Блок 8: Создание результатов для дашборда

In [86]:
# Блок 8: Создание результатов для дашборда

print("=== ПОДГОТОВКА ДАННЫХ ДЛЯ ДАШБОРДА ===")

# 1. Сводная таблица с метриками моделей
metrics_summary = spark.createDataFrame([
    ("Классификация", "AUC", auc),
    ("Классификация", "Accuracy", accuracy),
    ("Классификация", "F1-Score", f1),
    ("Классификация", "Precision", weightedPrecision),
    ("Классификация", "Recall", weightedRecall),
    ("Регрессия", "RMSE", rmse),
    ("Регрессия", "MAE", mae),
    ("Регрессия", "R²", r2),
    ("Регрессия", "MAPE", mape)
], ["model_type", "metric", "value"])

print("=== СВОДНАЯ ТАБЛИЦА МЕТРИК ===")
metrics_summary.show()

# 2. Таблица с примерами предсказаний
print("=== ПРИМЕРЫ ПРЕДСКАЗАНИЙ - КЛАССИФИКАЦИЯ ===")
sample_classifier_predictions = predictions_classifier.select(
    "has_tip", "prediction", "probability"
).limit(20)
sample_classifier_predictions.show()

print("=== ПРИМЕРЫ ПРЕДСКАЗАНИЙ - РЕГРЕССИЯ ===")
sample_regression_predictions = predictions_regression.select(
    "total_amount", "prediction",
    F.abs(F.col("prediction") - F.col("total_amount")).alias("error")
).limit(20)
sample_regression_predictions.show()

=== ПОДГОТОВКА ДАННЫХ ДЛЯ ДАШБОРДА ===
=== СВОДНАЯ ТАБЛИЦА МЕТРИК ===
+-------------+---------+------------------+
|   model_type|   metric|             value|
+-------------+---------+------------------+
|Классификация|      AUC|0.8423231494263405|
|Классификация| Accuracy|0.8439479260104983|
|Классификация| F1-Score|0.8363346140935409|
|Классификация|Precision|0.8525449791500384|
|Классификация|   Recall|0.8439479260104983|
|    Регрессия|     RMSE|  4.85003253489942|
|    Регрессия|      MAE| 2.782773332719316|
|    Регрессия|       R²|0.9331399000718417|
|    Регрессия|     MAPE| 9.984638642714877|
+-------------+---------+------------------+

=== ПРИМЕРЫ ПРЕДСКАЗАНИЙ - КЛАССИФИКАЦИЯ ===


25/11/17 21:35:00 WARN DAGScheduler: Broadcasting large task binary with size 1508.2 KiB
25/11/17 21:35:00 WARN DAGScheduler: Broadcasting large task binary with size 1508.2 KiB
                                                                                

+-------+----------+--------------------+
|has_tip|prediction|         probability|
+-------+----------+--------------------+
|      0|       1.0|[0.40243590998732...|
|      0|       1.0|[0.26651848380179...|
|      0|       0.0|[0.63413704028473...|
|      0|       0.0|[0.63413704028473...|
|      0|       0.0|[0.63413704028473...|
|      1|       1.0|[0.33744573306960...|
|      1|       1.0|[0.33744573306960...|
|      0|       1.0|[0.41356870559301...|
|      1|       1.0|[0.41356870559301...|
|      1|       0.0|[0.63413704028473...|
|      0|       0.0|[0.63413704028473...|
|      0|       0.0|[0.63413704028473...|
|      0|       1.0|[0.40243590998732...|
|      0|       1.0|[0.29396421616139...|
|      0|       1.0|[0.26651848380179...|
|      0|       1.0|[0.28049247733110...|
|      0|       1.0|[0.33104890091543...|
|      1|       1.0|[0.26949815459620...|
|      1|       1.0|[0.29396421616139...|
|      0|       1.0|[0.26557965708855...|
+-------+----------+--------------

[Stage 2807:==========================================>             (3 + 1) / 4]

+------------+------------------+-------------------+
|total_amount|        prediction|              error|
+------------+------------------+-------------------+
|       11.65| 14.33978122240012| 2.6897812224001196|
|       14.45|16.913286316765497| 2.4632863167654975|
|       11.65| 13.98311839539094| 2.3331183953909402|
|       11.65| 13.98311839539094| 2.3331183953909402|
|       11.65| 13.98311839539094| 2.3331183953909402|
|       18.85| 20.63465443848769| 1.7846544384876886|
|       21.54| 20.63465443848769| 0.9053455615123092|
|       27.71|13.782609791915558| 13.927390208084443|
|       13.95|13.782609791915558|0.16739020808444138|
|        13.1| 13.98311839539094|  0.883118395390941|
|       11.65| 13.98311839539094| 2.3331183953909402|
|       11.65| 13.98311839539094| 2.3331183953909402|
|       12.35| 14.33978122240012| 1.9897812224001203|
|       13.05|15.581588377334127|  2.531588377334126|
|       14.45|16.913286316765497| 2.4632863167654975|
|       16.55|27.03079932755

In [89]:
# 3. Важность признаков для классификации
print("=== ВАЖНОСТЬ ПРИЗНАКОВ - КЛАССИФИКАЦИЯ ===")
feature_importance_classifier = spark.createDataFrame(
    [(name, float(imp)) for name, imp in zip(assembler_inputs, gbt_model.featureImportances)],
    ["feature", "importance"]
).orderBy(F.col("importance").desc())

feature_importance_classifier.show(truncate=False)

# 4. Важность признаков для регрессии
print("=== ВАЖНОСТЬ ПРИЗНАКОВ - РЕГРЕССИЯ ===")
feature_importance_regression = spark.createDataFrame(
    [(name, float(imp)) for name, imp in zip(assembler_inputs, gbt_regressor_model.featureImportances)],
    ["feature", "importance"]
).orderBy(F.col("importance").desc())

feature_importance_regression.show(truncate=False)

=== ВАЖНОСТЬ ПРИЗНАКОВ - КЛАССИФИКАЦИЯ ===
+-----------------------+---------------------+
|feature                |importance           |
+-----------------------+---------------------+
|trip_distance          |0.006013003469929672 |
|is_weekend_encoded     |0.005663090608581261 |
|dropoff_borough_encoded|0.0044130926294553615|
|hour_encoded           |0.003492859811444092 |
|pickup_borough_encoded |0.0027754458445329545|
|day_of_week_encoded    |0.002622766519957862 |
|speed_mph              |0.0017516998983791068|
|trip_duration_minutes  |0.001605723848904222 |
|is_airport_trip_encoded|0.0012310299994716196|
|passenger_count        |8.819739893419079E-4 |
+-----------------------+---------------------+

=== ВАЖНОСТЬ ПРИЗНАКОВ - РЕГРЕССИЯ ===
+-----------------------+---------------------+
|feature                |importance           |
+-----------------------+---------------------+
|hour_encoded           |0.0029445743164887898|
|trip_duration_minutes  |0.0027526187925205193|
|day_

In [90]:
# 5. Анализ ошибок для регрессии
print("=== АНАЛИЗ ОШИБОК - РЕГРЕССИЯ ===")
error_analysis = predictions_regression.withColumn(
    "absolute_error", F.abs(F.col("prediction") - F.col("total_amount"))
).withColumn(
    "error_percentage", 
    F.when(F.col("total_amount") != 0, 
         F.abs(F.col("prediction") - F.col("total_amount")) / F.col("total_amount") * 100)
    .otherwise(0)
)

print("Топ-20 поездок с наибольшей ошибкой:")
error_analysis.orderBy(F.col("absolute_error").desc()).select(
    "total_amount", "prediction", "absolute_error", "error_percentage"
).limit(20).show()

=== АНАЛИЗ ОШИБОК - РЕГРЕССИЯ ===
Топ-20 поездок с наибольшей ошибкой:


[Stage 2810:=================================================>     (9 + 1) / 10]

+------------+------------------+-----------------+-----------------+
|total_amount|        prediction|   absolute_error| error_percentage|
+------------+------------------+-----------------+-----------------+
|      108.75|12.231757534023318|96.51824246597668|88.75240686526593|
|      108.34|13.022806871299817|95.31719312870018|87.97968721497155|
|       105.0|10.662647740093272|94.33735225990672|89.84509739038735|
|       108.3|14.046744797556514|94.25325520244348| 87.0297831970854|
|      105.22|11.103331975476383|94.11666802452362|89.44750810161909|
|      109.55|15.615149919760238|93.93485008023976|85.74609774554062|
|      104.94|11.025860975696572|93.91413902430342|89.49317612378829|
|       104.4|10.654743079284215|93.74525692071579|89.79430739532164|
|       104.7|11.070694590564848|93.62930540943515|89.42627068713959|
|       106.2|13.213545634003989|92.98645436599601|87.55786663464784|
|      106.15|13.337587640970774|92.81241235902922|87.43515059729555|
|       106.2|13.396

In [93]:
def save_to_postgres(data_frame, table_name):
    (data_frame.write.format("jdbc")
         .option("url", "jdbc:postgresql://postgres-db:5432/learn_base")
         .option("driver", "org.postgresql.Driver")
         .option("user", "airflow")
         .option("password", "airflow")
         .option("dbtable", f"nyc_taxi.{table_name}")
         .option("batchsize", 10000)
         .mode("overwrite")
         .save())

In [106]:
error_analysis.show()

[Stage 2832:==========================================>             (3 + 1) / 4]

+--------------------+------------+------------------+-------------------+------------------+
|            features|total_amount|        prediction|     absolute_error|  error_percentage|
+--------------------+------------+------------------+-------------------+------------------+
|(49,[0,23,29,36,4...|       11.65| 14.33978122240012| 2.6897812224001196| 23.08825083605253|
|(49,[0,23,29,36,4...|       14.45|16.913286316765497| 2.4632863167654975|17.046964129865035|
|(49,[0,23,29,36,4...|       11.65| 13.98311839539094| 2.3331183953909402|20.026767342411503|
|(49,[0,23,29,36,4...|       11.65| 13.98311839539094| 2.3331183953909402|20.026767342411503|
|(49,[0,23,29,36,4...|       11.65| 13.98311839539094| 2.3331183953909402|20.026767342411503|
|(49,[0,23,29,36,4...|       18.85| 20.63465443848769| 1.7846544384876886| 9.467662803648215|
|(49,[0,23,29,36,4...|       21.54| 20.63465443848769| 0.9053455615123092|  4.20308988631527|
|(49,[0,23,29,36,4...|       27.71|13.782609791915558| 13.92

In [ ]:
# Сохраняем результаты для дашборда
print("Сохранение результатов...")

In [99]:
save_to_postgres(metrics_summary, "results_metrics_summary")

In [105]:
save_to_postgres(sample_classifier_predictions.select(["has_tip", "prediction"]), "results_classifier_predictions")

25/11/17 21:46:38 WARN DAGScheduler: Broadcasting large task binary with size 1500.6 KiB
25/11/17 21:46:46 WARN DAGScheduler: Broadcasting large task binary with size 1430.2 KiB
                                                                                

In [101]:
save_to_postgres(sample_regression_predictions, "results_regression_predictions")

In [102]:
save_to_postgres(feature_importance_classifier, "results_feature_importance_classifier")

In [103]:
save_to_postgres(feature_importance_regression, "results_feature_importance_regression")

In [107]:
save_to_postgres(error_analysis.select(["total_amount", "prediction", "absolute_error", "error_percentage"]), "results_error_analysis")

print("Все результаты сохранены в папке 'results/'")

[Stage 2833:=================================================>     (9 + 1) / 10]

Все результаты сохранены в папке 'results/'


In [ ]:
# # Сохраняем результаты для дашборда
# print("Сохранение результатов...")
# metrics_summary.write.mode("overwrite").parquet("results/metrics_summary")
# sample_classifier_predictions.write.mode("overwrite").parquet("results/classifier_predictions")
# sample_regression_predictions.write.mode("overwrite").parquet("results/regression_predictions")
# feature_importance_classifier.write.mode("overwrite").parquet("results/feature_importance_classifier")
# feature_importance_regression.write.mode("overwrite").parquet("results/feature_importance_regression")
# error_analysis.write.mode("overwrite").parquet("results/error_analysis")

# print("Все результаты сохранены в папке 'results/'")

In [108]:
# (sample_classifier_predictions.write.format("jdbc")
#          .option("url", "jdbc:postgresql://postgres-db:5432/learn_base")
#          .option("driver", "org.postgresql.Driver")
#          .option("user", "airflow")
#          .option("password", "airflow")
#          .option("dbtable", "nyc_taxi.sample_classifier_predictions")
#          .option("batchsize", 10000)
#          .mode("overwrite")
#          .save())

In [109]:
# Блок 9: Интерпретация метрик (как читать результаты)

print("""
=== ИНТЕРПРЕТАЦИЯ МЕТРИК ===

ДЛЯ КЛАССИФИКАЦИИ (чаевые):

• AUC (Area Under ROC Curve): {:.4f}
  - 0.5 = модель не лучше случайного угадывания
  - 0.7-0.8 = хорошая модель
  - 0.8-0.9 = отличная модель
  - >0.9 = выдающаяся модель

• Accuracy: {:.4f}
  - Доля правильных предсказаний от общего числа
  - Может быть misleading при несбалансированных классах

• Precision: {:.4f}
  - Из всех предсказанных "чаевые будут", сколько действительно были с чаевыми?
  - "Насколько мы точны, когда предсказываем да"

• Recall: {:.4f}
  - Из всех реальных случаев с чаевыми, сколько мы правильно предсказали?
  - "Как много реальных чаевых мы нашли"

• F1-Score: {:.4f}
  - Гармоническое среднее между Precision и Recall
  - Хорошая общая метрика при несбалансированных классах

ДЛЯ РЕГРЕССИИ (стоимость):

• RMSE: {:.4f}
  - Средняя ошибка в долларах, но с усилением больших ошибок
  - В среднем модель ошибается на ±${:.2f}

• MAE: {:.4f}
  - Средняя абсолютная ошибка в долларах
  - Более интуитивная метрика чем RMSE

• R²: {:.4f}
  - Доля дисперсии, объясненная моделью
  - 0 = модель не объясняет ничего
  - 1 = идеальное предсказание
  - {:.1f}% дисперсии стоимости объясняется нашими признаками

• MAPE: {:.2f}%
  - Средняя процентная ошибка
  - В среднем модель ошибается на {:.1f}% от реальной стоимости

ВЫВОДЫ:
Классификация: {}
Регрессия: {}
""".format(
    auc, accuracy, weightedPrecision, weightedRecall, f1,
    rmse, rmse, mae, r2, r2*100, mape, mape,
    "Отличная модель" if auc > 0.8 else "Хорошая модель" if auc > 0.7 else "Требует улучшений",
    "Отличная модель" if r2 > 0.7 else "Хорошая модель" if r2 > 0.5 else "Требует улучшений"
))

# Завершаем работу
spark.stop()
print("Работа завершена!")


=== ИНТЕРПРЕТАЦИЯ МЕТРИК ===

ДЛЯ КЛАССИФИКАЦИИ (чаевые):

• AUC (Area Under ROC Curve): 0.8423
  - 0.5 = модель не лучше случайного угадывания
  - 0.7-0.8 = хорошая модель
  - 0.8-0.9 = отличная модель
  - >0.9 = выдающаяся модель

• Accuracy: 0.8439
  - Доля правильных предсказаний от общего числа
  - Может быть misleading при несбалансированных классах

• Precision: 0.8525
  - Из всех предсказанных "чаевые будут", сколько действительно были с чаевыми?
  - "Насколько мы точны, когда предсказываем да"

• Recall: 0.8439
  - Из всех реальных случаев с чаевыми, сколько мы правильно предсказали?
  - "Как много реальных чаевых мы нашли"

• F1-Score: 0.8363
  - Гармоническое среднее между Precision и Recall
  - Хорошая общая метрика при несбалансированных классах

ДЛЯ РЕГРЕССИИ (стоимость):

• RMSE: 4.8500
  - Средняя ошибка в долларах, но с усилением больших ошибок
  - В среднем модель ошибается на ±$4.85

• MAE: 2.7828
  - Средняя абсолютная ошибка в долларах
  - Более интуитивная метрик

In [10]:
spark.stop()